# 웹 크롤링에 의한 데이터 수집

## 오늘의 핵심강의 : '교보문고 주간 베스트' 크롤링 코드 이해 및 활용

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd

html = urlopen('http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf')
soup = bs(html, "html.parser")

book_page_urls = []
for cover in soup.find_all('div', {'class':'detail'}):
    link = cover.select('a')[0].get('href')
    book_page_urls.append(link)

dic = []
for index, book_page_url in enumerate(book_page_urls):
    html = urlopen(book_page_url)
    soup = bs(html, "html.parser")
    title = soup.find('meta', {'property':'og:title'}).get('content')
    author = soup.find('meta',{'name':'author'}).get('content')
    price = soup.find('meta', {'property':'og:price'}).get('content')
    dic.append({'Title':title,'Author' : author,'Price':price+'원'})

df = pd.DataFrame(dic)
df[4:]

,Title,Author,Price
0,달러구트 꿈 백화점. 2 - 교보문고,이미예,12420원
1,부의 시나리오 - 교보문고,오건영,16200원
2,밝은 밤 - 교보문고,최은영,13050원
3,설민석의 한국사 대모험. 17 - 교보문고,"설민석,",10800원
4,달러구트 꿈 백화점(50만부 기념 드림 에디션) - 교보문고,이미예,12420원
5,이상한 과자 가게 전천당. 11 - 교보문고,"히로시마 레이코 ,",10800원
6,미래의 부 - 교보문고,이지성,15300원
7,최악을 극복하는 힘 - 교보문고,"엘리자베스 스탠리 ,",24120원
8,해커스 토익 기출 보카 TOEIC VOCA - 교보문고,David Cho,11610원
9,강원국의 어른답게 말합니다 - 교보문고,강원국,14400원


# 1.HTML 에 대한 이해

## 가.HTML 문서의 구성 이해

In [ ]:
html_doc ='''<html><head><title>html연습</title></head>
<body>
<p class="title"><b><h1>HTML 연습</h1></b></p>
<p class="link">인터넷 링크를 작성하는 방법은 
<a class="kyobo" href='http://www.kyobobook.co.kr' id="link1">교보문고 홈페이지</a>과
<a class="python" href="https://www.python.org/" id="link2">파이썬 홈페이지</a>처럼 
HTML 링크로 작성할 수 있습니다.
</p>
<p class="story">그리고 HTML 문서는 글자의 크기도 조정 가능해서 
<h2>크게</h2>,<h4>작게</h4> 작성할 수 있습니다.
</p>
</body></html>
'''

In [ ]:
with open('html연습.html','w') as f:
  f.write(html_doc)

Html 요소 구조 : 태그이름, 시작태그, 속성명, 속성값, 내용, 종료태그

## 나.로봇 배제기준 확인

해당 홈페이지 명에 robots.txt를 추가시켜서 웹브라우져를 실행시키면 확인 가능

In [ ]:
# 예) https://news.naver.com/robots.txt  

# User-agent: Yeti
# Allow: /main/imagemontage
# Disallow: /
# User-agent: *
# Disallow: / 

이경우 오픈 api를 이용하여 서비스를 제공
https://openapi.naver.com/v1/search/news/

# 2.Requests 라이브러리를 이용한 웹페이지 정보 확인

참조 : https://requests.readthedocs.io/en/master/

## 가.웹페이지 정보를 간단하게 파악

In [ ]:
import requests
r = requests.get('http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf')
r.encoding

'EUC-KR'

In [ ]:
# 헤더 정보 확인
print(r.headers['Server'])
r.headers

Apache


{'Date': 'Fri, 06 Aug 2021 20:40:15 GMT', 'Server': 'Apache', 'Content-Language': 'ko-KR', 'Set-Cookie': 'WMONID=ROIrx6pbUid; expires=Saturday, 06-Aug-2022 20:40:15 GMT; path=/, KYOBOSESSIONID=hNpPb7CynVgcVBP2BdTDYpTpvHBP8f044MvFTjY888xczTynfMLn!-187090693!-2124087677!7200!-1!-1374614098!-2124087676!7200!-1; path=/, NSC_wjs_lzpcp_Dppljf=ffffffffd0b53b3045525d5f4f58455e445a4a423660;expires=Fri, 06-Aug-2021 20:42:16 GMT;path=/;httponly', 'Keep-Alive': 'timeout=5, max=49', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=EUC-KR', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'}

In [ ]:
# 텍스트 구조 파악
r.text

'\n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">\r\n<!-- s:html:head -->\r\n<head>\r\n\r\n\r\n\r\n\r\n\n\n\n\n\n\n\n\r\n \n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n\n\n\n\n\n\n\r\n\r\n\r\n<title>교보문고 종합 주간 집계 | 국내도서 | 베스트셀러 - 교보문고</title>\r\n\r\n\r\n\r\n\n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n<!--MS의 최신 웹브라우저인 edge 브라우저 호환을 위해 넣어줌-->\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr"/>\r\n<meta http-equiv="Pragma" content="nocache"/>\r\n<meta http-equiv="Expires" content="0"/>\r\n<meta http-equiv="Cache-Control" content="no-cache"/>\r\n\r\n\r\n\r\n<link rel="shortcut icon" href="http://image.kyobobook.co.kr/newimages/apps/b2c/kyobo.ICO"/>\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\

## 나.아이디와 패스를 입력해야하는 경우

In [ ]:
# 내 아이디 패스 저장
# %%writefile Mypass.py  
# naverid = 'abcd'
# naverpass = 'pass'

Writing Mypass.py


In [ ]:
import requests
from Mypass import * 

r = requests.get('https://blog.naver.com/enjoystat', auth=(naverid, naverpass))
print(r.headers['content-type'])
print(r.encoding)
print(r.text[:200])

text/html;charset=UTF-8
UTF-8







<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta http-equiv="Pragma" content="no-cache"/>



# 3.Urllib를 이용한 URL 접근

참조 : https://docs.python.org/ko/3.10/library/urllib.request.html

In [ ]:
# 식별된 encoding방법을 참고하여 decoding 및 출력, 바이트 객체를 반환함
import urllib.request
with urllib.request.urlopen('http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf') as f:
  print(f.read(500).decode('cp949')) # utf-8 또는 cp949(enc-kr)













<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">
<!-- s:html:head -->
<head>












 


































<title>교보문고 종합 주간 집계 | 국내도서 | 베스트셀러 - 교보문고</title>
















<!--MS의 최신 웹브라우저인 edge 브라우저 호환을 위해 넣어줌-->
<meta http-equiv="X-UA-


In [ ]:
import urllib.request
f = urllib.request.urlopen('http://www.python.org/')
content = f.headers.get_content_charset()
content


'utf-8'

In [ ]:
f.read(100).decode('utf-8')

'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!-'

# 4.Beautiful soup를 사용하여 웹페이지 내용 읽어 들이기

*웹페이지에서 소스크드를 보는 방법은 F12를 누르면 볼수 있다.

참조 : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

한글참조 : https://www.crummy.com/software/BeautifulSoup/bs4/doc.ko/

## 가.select와 find 사용법 차이

* select는 CSS 선택자 이용
* find는 태그와 속성, 속성값 이용
* find = select_one, find_all = select

In [ ]:
# find와 select_one 사용법 비교
from bs4 import BeautifulSoup as bs
tag = '''
<div class = 'section'>
<p class='start' id='s1'> 시작합니다! </p>
<p class='end' id='s2'> 마지막입니다! </p>
</div>
'''
soup = bs(tag) 

# find는 태그 이름, 속성으로 특정 
# print(soup.find('p')) 
# print(soup.find(class_='end')) 
# print(soup.find(attrs = {'class':'start'})) 
# print(soup.find('p', {'id':'s2'}))

# # select_one는 css로 특정
print(soup.select_one('p')) 
print(soup.select_one('.start')) 
print(soup.select_one('p.end')) 
print(soup.select_one('p#s1'))
print(soup.select_one('#s2'))

<p class="start" id="s1"> 시작합니다! </p>
<p class="start" id="s1"> 시작합니다! </p>
<p class="end" id="s2"> 마지막입니다! </p>
<p class="start" id="s1"> 시작합니다! </p>
<p class="end" id="s2"> 마지막입니다! </p>


In [ ]:
# find_all과 select 사용법 비교
from bs4 import BeautifulSoup as bs

soup = bs(tag) 

# print(soup.find('div').find('p').text)
# print(soup.select_one('div > #s2').text)
# for i in soup.find_all('p'):
#   print(i.get('id'))
for i in soup.select('p') :
  print(i.get('id'))

s1
s2


## 나.HTML문서 파싱과 데이터 정리

In [ ]:
from bs4 import BeautifulSoup as bs
soup = bs(html_doc, 'html.parser')  # 'lxml' 도 가능

print(soup.prettify())

<html>
 <head>
  <title>
   html연습
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    <h1>
     HTML 연습
    </h1>
   </b>
  </p>
  <p class="link">
   인터넷 링크를 작성하는 방법은
   <a class="kyobo" href="http://www.kyobobook.co.kr" id="link1">
    교보문고 홈페이지
   </a>
   과
   <a class="python" href="https://www.python.org/" id="link2">
    파이썬 홈페이지
   </a>
   처럼 
HTML 링크로 작성할 수 있습니다.
  </p>
  <p class="story">
   그리고 HTML 문서는 글자의 크기도 조정 가능해서
   <h2>
    크게
   </h2>
   ,
   <h4>
    작게
   </h4>
   작성할 수 있습니다.
  </p>
 </body>
</html>



In [ ]:
# 데이터 구조 식별하기, title 태그 내용 확인
print('1.', soup.title,'\n','2.', soup.title.parent.name)
soup.title.string

1. <title>html연습</title> 
 2. head


'html연습'

In [ ]:
# 태그로 식별 , 태그와 속성으로 속성값 출력
print('1.', soup.p,'\n', '2.', soup.p['class'])
soup.a

1. <p class="title"><b><h1>HTML 연습</h1></b></p> 
 2. ['title']


<a class="kyobo" href="http://www.kyobobook.co.kr" id="link1">교보문고 홈페이지</a>

In [ ]:
# select 함수는 리스트로 반환
for p in soup.select('p'):
    print(p.text)
# soup.select('p')

HTML 연습
인터넷 링크를 작성하는 방법은 
교보문고 홈페이지과
파이썬 홈페이지처럼 
HTML 링크로 작성할 수 있습니다.

그리고 HTML 문서는 글자의 크기도 조정 가능해서 
크게,작게 작성할 수 있습니다.



In [ ]:
# href 추출 
for link in soup.select('a'):
    print(link.get('href'))

http://www.kyobobook.co.kr
https://www.python.org/


In [ ]:
# 태그 내에 있는 속성값(get) 또는 내용(get_text) 추출, <a id='link2'> 같은 표현 숙지
# for p in soup.find_all('p'):
#     print(p['class'])

# s = soup.find('a',{'id':'link2'})
# print(s.get_text(), s.get('href'))

soup.find('p',{'class':'story'}).get_text()

'그리고 HTML 문서는 글자의 크기도 조정 가능해서 \n크게,작게 작성할 수 있습니다.\n'

In [ ]:
# 태그안에 있는 내용만 추출
import re
body=soup.body
text=body.get_text()
re.sub('\n','',text)

'HTML 연습인터넷 링크를 작성하는 방법은 교보문고 홈페이지과파이썬 홈페이지처럼 HTML 링크로 작성할 수 있습니다.그리고 HTML 문서는 글자의 크기도 조정 가능해서 크게,작게 작성할 수 있습니다.'

In [ ]:
# 내용을 리스트로 받아내기
text = []
for s in soup.find_all('p'):
  text.append(s.get_text())
text

['HTML 연습',
 '인터넷 링크를 작성하는 방법은 \n교보문고 홈페이지과\n파이썬 홈페이지처럼 \nHTML 링크로 작성할 수 있습니다.\n',
 '그리고 HTML 문서는 글자의 크기도 조정 가능해서 \n크게,작게 작성할 수 있습니다.\n']

In [ ]:
soup.find('p',{'class':'story'}).select('h2')[0].get_text()

'크게'

# 5.사례 연구 : 교보문고 주간 베스트

In [ ]:
# 필요한 라이브러리 선언, urlopen과 bs로 parsing
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd

html = urlopen('http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf')
soup = bs(html, "html.parser")

In [ ]:
# 주간 베스트에 해당하는 링크 url을 찾아서 리스트에 담기 
book_page_urls = []
for cover in soup.find_all('div', {'class':'detail'}):
    link = cover.select('a')[0].get('href')
    book_page_urls.append(link)
book_page_urls


['http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791165343729',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791165341909',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791190030922',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791191043297',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791190885928',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9788934985051',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791130640600',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791191056556',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791167370280',
 'http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791191824001',


In [ ]:
# url별로 순회하면서 딕셔너리 데이터로 담아서 list에 담기
dic = []
for index, book_page_url in enumerate(book_page_urls):
    html = urlopen(book_page_url)
    soup = bs(html, "html.parser")
    title = soup.find('meta', {'property':'og:title'}).get('content')
    author = soup.find('meta',{'name':'author'}).get('content')
    price = soup.find('meta', {'property':'og:price'}).get('content')
    # description = soup.find('meta', {'property':'og:description'}).get('content')
    # print(title, author, price+"원")
    dic.append({'Title':title,'Author' : author,'Price':price+'원'})



In [ ]:
# 데이터 확인
dic

[{'Author': '이미예', 'Price': '12420원', 'Title': '달러구트 꿈 백화점. 2 - 교보문고'},
 {'Author': '이미예', 'Price': '12420원', 'Title': '달러구트 꿈 백화점 - 교보문고'},
 {'Author': '¿¡¸¯ ¿ÍÀÌ³Ê ,',
  'Price': '16200원',
  'Title': '¼ÒÅ©¶óÅ×½º ÀÍ½ºÇÁ·¹½º - ±³º¸¹®°í'},
 {'Author': '이치조 미사키 ,',
  'Price': '12600원',
  'Title': '오늘 밤, 세계에서 이 사랑이 사라진다 해도 - 교보문고'},
 {'Author': '히가시노 게이고 ,', 'Price': '16200원', 'Title': '백조와 박쥐 - 교보문고'},
 {'Author': '장명숙',
  'Price': '13320원',
  'Title': '햇빛은 찬란하고 인생은 귀하니까요: 밀라논나 이야기 - 교보문고'},
 {'Author': '정지영', 'Price': '17100원', 'Title': '대한민국 재건축 재개발 지도 - 교보문고'},
 {'Author': '매트 헤이그 ,', 'Price': '14220원', 'Title': '미드나잇 라이브러리 - 교보문고'},
 {'Author': '정유정', 'Price': '14220원', 'Title': '완전한 행복 - 교보문고'},
 {'Author': '김초엽', 'Price': '13500원', 'Title': '지구 끝의 온실 - 교보문고'},
 {'Author': 'Eiichiro Oda',
  'Price': '4500원',
  'Title': '원피스. 99: 밀짚모자 루피 - 교보문고'},
 {'Author': '김호연', 'Price': '12600원', 'Title': '불편한 편의점 - 교보문고'},
 {'Author': '박세익', 'Price': '16200원', 'Title': '투자의 본질 - 교보문고'},
 {'Auth

In [ ]:
# 판다스 데이터 프레임으로 전환
pd.DataFrame(dic)

,Title,Author,Price
0,달러구트 꿈 백화점. 2 - 교보문고,이미예,12420원
1,달러구트 꿈 백화점 - 교보문고,이미예,12420원
2,¼ÒÅ©¶óÅ×½º ÀÍ½ºÇÁ·¹½º - ±³º¸¹®°í,"¿¡¸¯ ¿ÍÀÌ³Ê ,",16200원
3,"오늘 밤, 세계에서 이 사랑이 사라진다 해도 - 교보문고","이치조 미사키 ,",12600원
4,백조와 박쥐 - 교보문고,"히가시노 게이고 ,",16200원
5,햇빛은 찬란하고 인생은 귀하니까요: 밀라논나 이야기 - 교보문고,장명숙,13320원
6,대한민국 재건축 재개발 지도 - 교보문고,정지영,17100원
7,미드나잇 라이브러리 - 교보문고,"매트 헤이그 ,",14220원
8,완전한 행복 - 교보문고,정유정,14220원
9,지구 끝의 온실 - 교보문고,김초엽,13500원
